In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import keras
import os, sys
import matplotlib.pyplot as plt
import scipy.io
import pandas as pd
import numpy as np


# Define train data
columns = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class', 'fname']
train_images_dir = os.path.join('./', 'cars_train')
train_labels_mat = scipy.io.loadmat('cars_train_annos.mat')
train_labels_data = [[row.flat[0] for row in line] for line in train_labels_mat['annotations'][0]]
df_train = pd.DataFrame(train_labels_data, columns=columns)

# Define test data
columns = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'fname']
test_images_dir = os.path.join('./', 'cars_test')
test_labels_mat = scipy.io.loadmat('cars_test_annos.mat')
test_labels_data = [[row.flat[0] for row in line] for line in test_labels_mat['annotations'][0]]
df_test = pd.DataFrame(test_labels_data, columns=columns) 

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

img_width, img_height = 256, 256
nb_train_samples = 6114
nb_validation_samples = 2000
batch_size = 15
epochs = 100

model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))


# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers[:5]:
    layer.trainable = False

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(196, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

Using TensorFlow backend.
/home/samuel/.virtualenvs/ai/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [ ]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg19_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

logger = keras.callbacks.TensorBoard(log_dir="logs", write_graph=True, histogram_freq=5)


In [ ]:
# Initiate the train and test generators with data Augumentation 
train_image_generator = ImageDataGenerator(
                            rescale=1./255,
                        )

test_image_generator = ImageDataGenerator(rescale=1./255)

train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_images_dir,
                                                           target_size=(img_height, img_width),
                                                           shuffle=True,
                                                           class_mode='categorical')

test_data_gen = test_image_generator.flow_from_directory(batch_size=batch_size,
                                                             directory=test_images_dir,
                                                             target_size=(img_height, img_width),
                                                             class_mode='categorical')

# Train the model 
model_final.fit_generator(
train_data_gen,
steps_per_epoch=6114 // batch_size,
epochs = epochs,
validation_data = test_data_gen,
validation_steps=2000 // batch_size,
callbacks = [checkpoint, early, logger])

Found 6144 images belonging to 196 classes.
Found 2000 images belonging to 196 classes.
Epoch 1/100
407/407 [==============================] - 58s 143ms/step - loss: 5.2757 - accuracy: 0.0067 - val_loss: 5.2862 - val_accuracy: 0.0090
Epoch 2/100
407/407 [==============================] - 58s 143ms/step - loss: 5.2757 - accuracy: 0.0084 - val_loss: 5.2832 - val_accuracy: 0.0081
Epoch 3/100
407/407 [==============================] - 58s 143ms/step - loss: 5.2755 - accuracy: 0.0064 - val_loss: 5.2840 - val_accuracy: 0.0086
Epoch 4/100
407/407 [==============================] - 59s 144ms/step - loss: 5.2722 - accuracy: 0.0052 - val_loss: 5.2693 - val_accuracy: 0.0081
Epoch 5/100
407/407 [==============================] - 59s 144ms/step - loss: 5.2711 - accuracy: 0.0077 - val_loss: 5.2669 - val_accuracy: 0.0091
Epoch 6/100
407/407 [==============================] - 59s 145ms/step - loss: 5.2667 - accuracy: 0.0089 - val_loss: 5.2806 - val_accuracy: 0.0091
Epoch 7/100
407/407 [===============

## Try out image augmentation, l2 regularization

In [ ]:
from keras import regularizers

model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

checkpoint = ModelCheckpoint("vgg19_2.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

logger = keras.callbacks.TensorBoard(log_dir="vgg19_2", write_graph=True, histogram_freq=5)

# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers[:5]:
    layer.trainable = False

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, kernel_regularizer=regularizers.l2(0.1), activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, kernel_regularizer=regularizers.l2(0.1), activation="relu")(x)
predictions = Dense(196, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

# Initiate the train and test generators with data Augumentation 
train_image_generator = ImageDataGenerator(
                            rescale=1./255,
                            rotation_range=45,
                            width_shift_range=.15,
                            height_shift_range=.15,
                            horizontal_flip=True,
                            shear_range=0.2,
                            zoom_range=0.3
                        )

test_image_generator = ImageDataGenerator(rescale=1./255)

train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_images_dir,
                                                           target_size=(img_height, img_width),
                                                           shuffle=True,
                                                           class_mode='categorical')

test_data_gen = test_image_generator.flow_from_directory(batch_size=batch_size,
                                                             directory=test_images_dir,
                                                             target_size=(img_height, img_width),
                                                             class_mode='categorical')


# Train the model 
model_final.fit_generator(
train_data_gen,
steps_per_epoch=6114 // batch_size,
epochs = epochs,
validation_data = test_data_gen,
validation_steps=2000 // batch_size,
callbacks = [checkpoint, early, logger])

/home/samuel/.virtualenvs/ai/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 6144 images belonging to 196 classes.
Found 2000 images belonging to 196 classes.
Epoch 1/100
407/407 [==============================] - 75s 184ms/step - loss: 1058.9792 - accuracy: 0.0049 - val_loss: 975.2362 - val_accuracy: 0.0030


/home/samuel/.virtualenvs/ai/lib/python3.6/site-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
/home/samuel/.virtualenvs/ai/lib/python3.6/site-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 2/100
407/407 [==============================] - 71s 173ms/step - loss: 900.4309 - accuracy: 0.0066 - val_loss: 829.2509 - val_accuracy: 0.0091
Epoch 3/100
407/407 [==============================] - 70s 173ms/step - loss: 765.7074 - accuracy: 0.0072 - val_loss: 705.2344 - val_accuracy: 0.0111
Epoch 4/100
407/407 [==============================] - 70s 172ms/step - loss: 651.2600 - accuracy: 0.0072 - val_loss: 599.8989 - val_accuracy: 0.0086
Epoch 5/100
407/407 [==============================] - 73s 179ms/step - loss: 554.0374 - accuracy: 0.0062 - val_loss: 510.4167 - val_accuracy: 0.0086
Epoch 6/100
407/407 [==============================] - 71s 174ms/step - loss: 471.4469 - accuracy: 0.0059 - val_loss: 434.3623 - val_accuracy: 0.0091
Epoch 7/100
407/407 [==============================] - 72s 176ms/step - loss: 401.2867 - accuracy: 0.0080 - val_loss: 369.8246 - val_accuracy: 0.0081
Epoch 8/100
407/407 [==============================] - 73s 179ms/step - loss: 341.6854 - accuracy: 0